In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from numpy import genfromtxt
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import pandas as pd
from sklearn import preprocessing

In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/data.csv')
# data = data[data['price'] != 0]

In [ ]:
# data=data[data["price"]/data["sqft_living"]<10000]

In [ ]:
x = data["price"]/data["sqft_living"]
plt.hist(x)

In [ ]:
data['date'] = pd.factorize(data['date'])[0] + 0.0
data['street'] = pd.factorize(data['street'])[0] +0.0
data['city'] = pd.factorize(data['city'])[0] + 0.0
data['statezip'] = pd.factorize(data['statezip'])[0] + 0.0
data['country'] = pd.factorize(data['country'])[0] + 0.0

In [ ]:
y = data.price
x = data.drop('price' , axis = 1)
# x = data.drop(['country','price'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 50)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state = 20)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
X_val = scaler.fit_transform(X_val)

In [ ]:
def build_model():
    model = Sequential()
    model.add(Dense(256, activation='relu',input_shape=[17]))
    model.add(Dense(128, activation= 'relu'))
    model.add(Dense(64, activation= 'relu'))

    model.add(Dense(1, activation= 'linear'))
    model.compile(optimizer='adam', loss='mae', metrics=['mae'])
    return model
model = build_model()
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
recordModel = model.fit(X_train, y_train, epochs=500,validation_data = (X_val,y_val ),callbacks=[callback])

In [ ]:
fig_Loss = plt.figure(figsize=(10, 10))
plt.plot(recordModel.history['loss'])
plt.plot(recordModel.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train Loss","Validation Loss"])
plt.show()

In [ ]:
Test_Predict = model.predict(X_test)
plt.scatter(y_test,Test_Predict)
_ = plt.plot([0, 2.5e6],[0, 2.5e6], color = 'red')
plt.xlabel('Right Values')
plt.ylabel('Prediction')
plt.show()
mae = model.evaluate(X_test, y_test)[0]
print("Test - Mean Absoulate Error " ,mae)

In [ ]:
class RidgeRegression() :
	
	def __init__( self, learning_rate, iterations, l2_penality ) :
		
		self.learning_rate = learning_rate		
		self.iterations = iterations		
		self.l2_penality = l2_penality
					
	def fit( self, X, Y ) :
		
		self.m, self.n = X.shape
				
		self.W = np.zeros( self.n )
		
		self.b = 0		
		self.X = X		
		self.Y = Y
				
		for i in range( self.iterations ) :			
			self.update_weights()			
		return self
	
	def update_weights( self ) :		
		Y_pred = self.predict( self.X )
		
		dW = ( - ( 2 * ( self.X.T ).dot( self.Y - Y_pred ) ) + ( 2 * self.l2_penality * self.W ) ) / self.m	
		db = - 2 * np.sum( self.Y - Y_pred ) / self.m
		
		
		self.W = self.W - self.learning_rate * dW	
		self.b = self.b - self.learning_rate * db		
		return self

	def predict( self, X ) :	
		return X.dot( self.W ) + self.b
		
model = RidgeRegression( iterations = 1000, learning_rate = 0.01, l2_penality = 1 )
model.fit( X_train, y_train )


Y_pred = model.predict( X_test )
Test_Predict = model.predict(X_test)
plt.scatter(y_test,Test_Predict)
_ = plt.plot([0, 2.5e6],[0, 2.5e6], color = 'red')
plt.xlabel('Right Values')
plt.ylabel('Prediction')
plt.show()
